In [22]:
import os
%pwd

'd:\\2025-courses\\mlops\\TextSummarizer'

In [23]:
#os.chdir('../')
%pwd

'd:\\2025-courses\\mlops\\TextSummarizer'

In [24]:
from dataclasses import dataclass, field
from pathlib import Path

# now for model trainer config, we should read params.yaml config file
@dataclass
class ModelTrainerConfig:
    root_dir:Path
    data_path:Path
    model_ckpt:Path
    num_train_epochs:int
    warmup_steps:int
    per_device_train_batch_size:int
    per_device_eval_batch_size:int
    weight_decay:float
    logging_steps:int
    evaluation_strategy:str
    eval_steps:int
    save_steps:int
    gradient_accumulation_steps:int


In [25]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManger:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])


        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

In [28]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        #load tokeniser
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        #load model
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            eval_strategy = self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(
            model = model_pegasus,
            args = trainer_args,
            data_collator = seq2seq_data_collector,
            train_dataset = dataset_samsum_pt["train"],
            eval_dataset = dataset_samsum_pt["validation"],
            tokenizer = tokenizer
        )

        # Start training
        trainer.train()

        #save  the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))




In [ ]:
config = ConfigurationManger()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(model_trainer_config)
model_trainer.train()

[2025-09-28 09:06:38,198: INFO:  common: yaml file: config\config.yaml loaded successfully]
[2025-09-28 09:06:38,215: INFO:  common: yaml file: params.yaml loaded successfully]
[2025-09-28 09:06:38,217: INFO:  common: created directory at: artifacts]
[2025-09-28 09:06:38,218: INFO:  common: created directory at: artifacts/model_trainer]
[2025-09-28 09:06:38,215: INFO:  common: yaml file: params.yaml loaded successfully]
[2025-09-28 09:06:38,217: INFO:  common: created directory at: artifacts]
[2025-09-28 09:06:38,218: INFO:  common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\pc\AppData\Local\Temp\ipykernel_35856\2700159835.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\pc\AppData\Local\Temp\ipykernel_35856\2700159835.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated 

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda'